In [ ]:
import os
import ast
import matplotlib.pyplot as plt
import sys
#sys.path.append(os.path.join(os.getcwd(), '../desi/'))
#import catalog_definitions as cat
import numpy as np
import emcee
import time
from multiprocessing import Pool
import corner

In [ ]:
def log_prob(x, mu, cov):
    diff = x - mu
    return -0.5 * np.dot(diff, np.linalg.solve(cov, diff))

ndim = 10

np.random.seed(42)

 # Fixed values, not parameters.
means = np.random.rand(ndim) # Means are between 0 and 1
# There are some covariancse between the 5 gaussians
cov = 0.5 - np.random.rand(ndim**2).reshape((ndim, ndim)) 
cov = np.triu(cov)
cov += cov.T - np.diag(cov.diagonal())
cov = np.dot(cov, cov)

nwalkers = 32
niter = 10000
nburnin = 1000

In [ ]:
print(means)

In [ ]:
p0 = np.random.uniform(low=np.zeros(ndim), high=np.ones(ndim), size=(nwalkers, ndim))

In [ ]:
backend = emcee.backends.HDFBackend("test_backend")

In [ ]:

# -- run emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[means, cov], backend=backend)
#state = sampler.run_mcmc(p0, nburnin)
#sampler.reset()

start = time.time()
# If you run this multiple times, it will keep doing 10000 more steps (per walker)
sampler.run_mcmc(p0, niter, progress=True)
end = time.time()
serial_time = end - start

print("Serial took {0:.1f} seconds".format(serial_time))

In [ ]:
samples = sampler.get_chain()
ndim = samples.shape[2]
print(f'Number of steps: {samples.shape[0] * samples.shape[1]} (total); {samples.shape[0]} (per walker), ')
print(f'Number of walkers: {samples.shape[1]}')
print(f'Number of parameters: {ndim}')

try:
    tau = sampler.get_autocorr_time()
    print(tau)
except:
    print("Not burnt in yet")

burn = int(np.max(tau) * 2)
flatchain = sampler.get_chain(discard=burn, flat=True)

fig = corner.corner(
    flatchain
)

In [ ]:
with Pool() as pool:
    sampler2 = emcee.EnsembleSampler(nwalkers, ndim, log_prob, pool=pool, args=[means, cov])
    state2 = sampler2.run_mcmc(p0, nburnin)
    sampler2.reset()

    start = time.time()
    sampler2.run_mcmc(p0, niter, progress=True)
    end = time.time()
    multi_time = end - start
    
    print("Multiprocessing took {0:.1f} seconds".format(multi_time))
    print("{0:.1f} times faster than serial".format(serial_time / multi_time))


In [ ]:
import matplotlib.pyplot as plt

samples = sampler.get_chain(flat=True) # length is walkers * steps run total
PARAMETER_INDEX = 0

# View distribution of values (not including burn in) of a single parameter
plt.hist(samples[:, PARAMETER_INDEX], 100, color="k", histtype="step")
plt.xlabel(r"$\theta_1$")
plt.ylabel(r"$p(\theta_1)$")
plt.title(f'Posterior Distribution for parameter {PARAMETER_INDEX}')
plt.gca().set_yticks([])
plt.show()

# View chain for a single parameter
plt.plot(samples[:, PARAMETER_INDEX])
plt.xlabel('Iteration')
plt.ylabel(f'Parameter Value')
plt.title(f'Parameter Chain for parameter {PARAMETER_INDEX}')
plt.show()

In [ ]:
print(
    "Mean acceptance fraction: {0:.3f}".format(
        np.mean(sampler.acceptance_fraction)
    )
)
print(
    "Mean autocorrelation time: {0:.3f} steps".format(
        np.mean(sampler.get_autocorr_time())
    )
)

In [ ]:
# Corner Plots
import corner

fig = corner.corner(
    flat_samples, labels=labels, truths=[m_true, b_true, np.log(f_true)]
);